In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

rawData = pd.read_csv('data.csv')
infoData = pd.DataFrame()
infoData['FLAG'] = rawData['FLAG']
infoData['CONS_NO'] = rawData['CONS_NO']
data = rawData.drop(['FLAG', 'CONS_NO'], axis=1)

dropIndex = data[data.duplicated()].index  # duplicates drop
data = data.drop(dropIndex, axis=0)
infoData = infoData.drop(dropIndex, axis=0)

zeroIndex = data[(data.sum(axis=1) == 0)].index  # zero rows drop
data = data.drop(zeroIndex, axis=0)
infoData = infoData.drop(zeroIndex, axis=0)

data.columns = pd.to_datetime(data.columns)  # columns reindexing according to dates
data = data.reindex(sorted(data.columns), axis=1)
cols = data.columns

data.reset_index(inplace=True, drop=True)  # index sorting
infoData.reset_index(inplace=True, drop=True)

data = data.interpolate(method='linear', limit=2,  # filling NaN values
                        limit_direction='both', axis=0).fillna(0)

for i in range(data.shape[0]):  # outliers treatment
    m = data.loc[i].mean()
    st = data.loc[i].std()
    data.loc[i] = data.loc[i].mask(data.loc[i] > (m + 3 * st), other=m + 3 * st)

data.to_csv(r'visualization.csv', index=False, header=True)  # preprocessed data without scaling

scale = MinMaxScaler()
scaled = scale.fit_transform(data.values.T).T
mData = pd.DataFrame(data=scaled, columns=data.columns)
print(mData)
preprData = pd.concat([infoData, mData], axis=1, sort=False)  # Back to initial format
print(preprData)
preprData.to_csv(r'preprocessedR.csv', index=False, header=True)

       2014-01-01  2014-01-02  2014-01-03  2014-01-04  2014-01-05  2014-01-06  \
0        0.000000    0.000000    0.000000    0.000000    0.000000    0.000000   
1        0.108235    0.210498    0.260883    0.123910    0.134734    0.199674   
2        0.077443    0.150614    0.186665    0.088659    0.096404    0.142869   
3        0.111323    0.216503    0.268326    0.127445    0.138577    0.205371   
4        0.141255    0.269839    0.339762    0.168473    0.179267    0.260453   
...           ...         ...         ...         ...         ...         ...   
40251    0.330066    0.156642    0.152912    0.507221    0.798437    0.694631   
40252    0.275157    0.000000    0.000000    0.582925    0.616556    0.592097   
40253    0.078565    0.157130    0.124620    0.132748    0.208604    0.186931   
40254    0.417892    0.374841    0.477026    0.425315    0.415666    0.432490   
40255    0.444454    0.398666    0.507346    0.452348    0.442086    0.459979   

       2014-01-07  2014-01-